In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import streamlit as st
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Lipinski, MolSurf, QED, rdMolDescriptors
import warnings
warnings.filterwarnings("ignore")

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# Custom color palette for visualizations
colors = {
    'primary': '#1f77b4',
    'secondary': '#ff7f0e',
    'tertiary': '#2ca02c',
    'quaternary': '#d62728',
    'highlight': '#9467bd'
}

# Set random seed for reproducibility
np.random.seed(42)

In [25]:
# # Generate submission
# submission = pd.DataFrame({'Batch_ID': df_test['Batch_ID'], 'T80': y_pred_ensemble})
# submission.to_csv('submission21.csv', index=False)
# print("Submission saved to submission2.csv")

In [4]:
def load_data(train_path, test_path):
    """
    Load training and test datasets
    """
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)
    submission = pd.read_csv("../molecular-machine-learning/data/sample_submission.csv")
    
    print(f"Training data shape: {df_train.shape}")
    print(f"Test data shape: {df_test.shape}")
    
    return df_train, df_test, submission

In [5]:
# Function for data exploration
def explore_data(df):
    """
    Perform initial data exploration
    """
    print("\n==== Data Overview ====")
    print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns")
    
    # Data types and missing values
    data_types = pd.DataFrame({
        'Type': df.dtypes,
        'Non-Null Count': df.count(),
        'Missing Values': df.isnull().sum(),
        'Missing Percentage': (df.isnull().sum() / len(df) * 100).round(2)
    })
    
    print("\n==== Data Types and Missing Values ====")
    print(data_types[data_types['Missing Values'] > 0])
    
    # Basic statistics
    print("\n==== Numerical Features Statistics ====")
    print(df.describe().T)
    
    return data_types

In [6]:
# Function for data cleaning
def clean_data(df_train, df_test):
    """
    Clean the datasets by handling missing values and outliers
    """
    print("\n==== Data Cleaning ====")
    
    # Create copies to avoid modifying original data
    train_clean = df_train.copy()
    test_clean = df_test.copy()
    
    # Handle missing values
    for column in train_clean.select_dtypes(include=[np.number]).columns:
        if train_clean[column].isnull().any():
            median_val = train_clean[column].median()
            train_clean[column] = train_clean[column].fillna(median_val)
            test_clean[column] = test_clean[column].fillna(median_val)
            print(f"Filled missing values in {column} with median: {median_val}")
    
    # Handle outliers for key features
    key_features = ['T80', 'Mass', 'NumHeteroatoms', 'TDOS4.0']
    for feature in key_features:
        if feature in train_clean.columns:
            Q1 = train_clean[feature].quantile(0.25)
            Q3 = train_clean[feature].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR  # Less aggressive outlier removal
            upper_bound = Q3 + 3 * IQR
            
            # Count outliers
            outliers_count = ((train_clean[feature] < lower_bound) | (train_clean[feature] > upper_bound)).sum()
            
            if outliers_count > 0:
                print(f"Capping {outliers_count} outliers in {feature}")
                train_clean[feature] = train_clean[feature].clip(lower_bound, upper_bound)
                if feature != 'T80':  # Don't cap T80 in test set as it's the target
                    test_clean[feature] = test_clean[feature].clip(lower_bound, upper_bound)
    
    return train_clean, test_clean

In [7]:
# Function for feature engineering using RDKit
def engineer_rdkit_features(df_train, df_test):
    """
    Engineer additional molecular features using RDKit
    """
    print("\n==== RDKit Feature Engineering ====")
    
    def compute_rdkit_features(smiles):
        """Calculate RDKit molecular descriptors from SMILES string"""
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
            
            features = {
                # Basic properties
                'RDKit_MolWt': Descriptors.MolWt(mol),
                'RDKit_LogP': Descriptors.MolLogP(mol),
                'RDKit_TPSA': Descriptors.TPSA(mol),
                
                # Electronic properties
                'RDKit_NumValenceElectrons': Descriptors.NumValenceElectrons(mol),
                'RDKit_NumRadicalElectrons': Descriptors.NumRadicalElectrons(mol),
                
                # Lipinski properties
                'RDKit_HBD': Lipinski.NumHDonors(mol),
                'RDKit_HBA': Lipinski.NumHAcceptors(mol),
                'RDKit_RotBonds': Lipinski.NumRotatableBonds(mol),
                
                # Topological properties
                'RDKit_Rings': rdMolDescriptors.CalcNumRings(mol),
                'RDKit_AromaticRings': rdMolDescriptors.CalcNumAromaticRings(mol),
                'RDKit_HeteroRings': rdMolDescriptors.CalcNumHeteroRings(mol),
                'RDKit_Sp3Atoms': rdMolDescriptors.CalcFractionCSP3(mol),
                
                # Surface and volume properties
                'RDKit_LabuteASA': rdMolDescriptors.CalcLabuteASA(mol),
                'RDKit_PEOE_VSA': sum(rdMolDescriptors.PEOE_VSA_(mol)),
                'RDKit_SMR_VSA': sum(rdMolDescriptors.SMR_VSA_(mol)),
                
                # QED and related
                'RDKit_QED': QED.qed(mol),
                
                # Morgan fingerprints (ECFP) - aggregated properties
                'RDKit_ECFP4_Count': len(AllChem.GetMorganFingerprint(mol, 2).GetNonzeroElements()),
                'RDKit_ECFP4_Complexity': sum(AllChem.GetMorganFingerprint(mol, 2).GetNonzeroElements().values()),
                
                # Additional electronic properties
                'RDKit_PEOE_VSA_PosSum': sum([x for x in rdMolDescriptors.PEOE_VSA_(mol) if x > 0]),
                'RDKit_PEOE_VSA_NegSum': sum([x for x in rdMolDescriptors.PEOE_VSA_(mol) if x < 0]),
            }
            
            return pd.Series(features)
        except:
            print(f"Error processing SMILES: {smiles}")
            return None
    
    # Apply feature engineering to both datasets
    train_rdkit = df_train['Smiles'].apply(compute_rdkit_features)
    test_rdkit = df_test['Smiles'].apply(compute_rdkit_features)
    
    # Check for invalid SMILES
    invalid_train = train_rdkit.isna().all(axis=1)
    invalid_test = test_rdkit.isna().all(axis=1)
    
    if invalid_train.any():
        print(f"Found {invalid_train.sum()} invalid SMILES in training data, removing them")
        valid_train_idx = ~invalid_train
        df_train = df_train.loc[valid_train_idx].reset_index(drop=True)
        train_rdkit = train_rdkit.loc[valid_train_idx].reset_index(drop=True)
    
    if invalid_test.any():
        print(f"Found {invalid_test.sum()} invalid SMILES in test data, removing them")
        valid_test_idx = ~invalid_test
        df_test = df_test.loc[valid_test_idx].reset_index(drop=True)
        test_rdkit = test_rdkit.loc[valid_test_idx].reset_index(drop=True)
    
    # Merge RDKit features with original data
    df_train_enhanced = pd.concat([df_train, train_rdkit], axis=1)
    df_test_enhanced = pd.concat([df_test, test_rdkit], axis=1)
    
    print(f"Added {train_rdkit.shape[1]} RDKit features")
    
    return df_train_enhanced, df_test_enhanced

In [8]:
# Function for data visualization
def visualize_data(df_train):
    """
    Create visualizations for data analysis
    """
    print("\n==== Data Visualization ====")
    
    # 1. Target variable distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(df_train['T80'], kde=True, color=colors['primary'])
    plt.title('Distribution of T80 (Photostability Lifetime)', fontsize=14)
    plt.xlabel('T80 Value', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.savefig('t80_distribution.png', dpi=300, bbox_inches='tight')
    
    # 2. Correlation between key features and target
    key_features = ['TDOS4.0', 'NumHeteroatoms', 'Mass']
    plt.figure(figsize=(18, 5))
    
    for i, feature in enumerate(key_features, 1):
        plt.subplot(1, 3, i)
        sns.scatterplot(x=df_train[feature], y=df_train['T80'], color=colors['secondary'], alpha=0.7)
        
        # Add regression line
        sns.regplot(x=df_train[feature], y=df_train['T80'], scatter=False, color=colors['quaternary'])
        
        plt.title(f'{feature} vs T80', fontsize=12)
        plt.xlabel(feature, fontsize=10)
        plt.ylabel('T80', fontsize=10)
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('key_features_correlation.png', dpi=300, bbox_inches='tight')
    
    # 3. Correlation heatmap
    # Select numerical columns excluding ID columns and SMILES
    numerical_cols = df_train.select_dtypes(include=[np.number]).columns
    numerical_cols = [col for col in numerical_cols if 'ID' not in col]
    
    # Calculate correlations
    corr_matrix = df_train[numerical_cols].corr()
    
    # Focus on correlations with T80
    t80_corr = corr_matrix['T80'].sort_values(ascending=False)
    
    # Top 15 correlations with T80
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        corr_matrix.loc[t80_corr.index[:15], t80_corr.index[:15]], 
        annot=True, 
        cmap='viridis', 
        linewidths=0.5, 
        fmt=".2f",
        annot_kws={"size": 8}
    )
    plt.title('Correlation Heatmap: Top 15 Features by T80 Correlation', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
    
    # 4. PCA Visualization (2D projection of molecules)
    # Select numerical features
    feature_cols = [col for col in df_train.select_dtypes(include=[np.number]).columns 
                   if col != 'T80' and 'ID' not in col]
    
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_train[feature_cols])
    
    # Apply PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(scaled_features)
    
    # Create DataFrame for plotting
    pca_df = pd.DataFrame({
        'PC1': pca_result[:, 0],
        'PC2': pca_result[:, 1],
        'T80': df_train['T80']
    })
    
    # Plot PCA results
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(
        pca_df['PC1'], 
        pca_df['PC2'], 
        c=pca_df['T80'], 
        cmap='viridis',
        alpha=0.7,
        s=50
    )
    plt.colorbar(scatter, label='T80 Value')
    plt.title('PCA: 2D Projection of Molecules Colored by T80', fontsize=14)
    plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
    plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.savefig('pca_visualization.png', dpi=300, bbox_inches='tight')
    
    # 5. Pairplot of key features
    key_features_with_target = key_features + ['T80', 'RDKit_TPSA', 'RDKit_QED'] 
    key_features_with_target = [f for f in key_features_with_target if f in df_train.columns]
    
    plt.figure(figsize=(15, 12))
    sns.pairplot(
        df_train[key_features_with_target], 
        diag_kind='kde',
        plot_kws={'alpha': 0.6, 'color': colors['primary']},
        diag_kws={'color': colors['secondary']}
    )
    plt.suptitle('Pairplot of Key Features', y=1.02, fontsize=16)
    plt.savefig('key_features_pairplot.png', dpi=300, bbox_inches='tight')
    
    print("Visualizations saved as PNG files")
    
    return {
        't80_distribution.png': 'Distribution of target variable',
        'key_features_correlation.png': 'Correlation between key features and target',
        'correlation_heatmap.png': 'Correlation heatmap of top features',
        'pca_visualization.png': 'PCA visualization of molecules',
        'key_features_pairplot.png': 'Pairplot of key features'
    }

In [9]:
# Function for feature selection
def select_features(X_train, y_train, X_test, method='combined'):
    """
    Select important features using multiple approaches
    """
    print("\n==== Feature Selection ====")
    
    # Ensure key features are included
    key_features = ['TDOS4.0', 'NumHeteroatoms', 'Mass']
    key_features_present = [f for f in key_features if f in X_train.columns]
    
    if method == 'random_forest':
        # Random Forest feature importance
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        
        # Get feature importances
        importances = rf.feature_importances_
        feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)
        
        # Select top features
        top_features = feature_importance.head(20)['Feature'].tolist()
        
        # Make sure key features are included
        for feature in key_features_present:
            if feature not in top_features:
                top_features.append(feature)
                
        print(f"Selected {len(top_features)} features using Random Forest Importance")
        
    elif method == 'rfe':
        # Recursive Feature Elimination
        estimator = SVR(kernel='linear')
        selector = RFECV(estimator, step=1, cv=5, min_features_to_select=10, scoring='neg_mean_squared_error')
        selector.fit(X_train, y_train)
        
        # Get selected features
        selected_features_mask = selector.support_
        top_features = X_train.columns[selected_features_mask].tolist()
        
        # Make sure key features are included
        for feature in key_features_present:
            if feature not in top_features:
                top_features.append(feature)
                
        print(f"Selected {len(top_features)} features using RFE with cross-validation")
        
    elif method == 'combined':
        # Combine multiple feature selection methods
        
        # Method 1: Random Forest Importance
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        
        rf_importance = pd.DataFrame({
            'Feature': X_train.columns,
            'RF_Importance': rf.feature_importances_
        }).sort_values('RF_Importance', ascending=False)
        
        # Method 2: Lasso Coefficient
        lasso = Lasso(alpha=0.01, random_state=42)
        lasso.fit(X_train, y_train)
        
        lasso_importance = pd.DataFrame({
            'Feature': X_train.columns,
            'Lasso_Coef': np.abs(lasso.coef_)
        }).sort_values('Lasso_Coef', ascending=False)
        
        # Method 3: Correlation with target
        corr_with_target = pd.DataFrame({
            'Feature': X_train.columns,
            'Correlation': [abs(np.corrcoef(X_train[col], y_train)[0, 1]) for col in X_train.columns]
        }).sort_values('Correlation', ascending=False)
        
        # Combine rankings
        feature_ranks = pd.DataFrame({'Feature': X_train.columns})
        
        # Add rankings from each method
        feature_ranks = feature_ranks.merge(
            rf_importance[['Feature', 'RF_Importance']], 
            on='Feature', 
            how='left'
        ).merge(
            lasso_importance[['Feature', 'Lasso_Coef']], 
            on='Feature', 
            how='left'
        ).merge(
            corr_with_target[['Feature', 'Correlation']], 
            on='Feature', 
            how='left'
        )
        
        # Create a combined score (normalized sum of all methods)
        for col in ['RF_Importance', 'Lasso_Coef', 'Correlation']:
            feature_ranks[f'{col}_Norm'] = feature_ranks[col] / feature_ranks[col].max()
        
        feature_ranks['Combined_Score'] = (
            feature_ranks['RF_Importance_Norm'] + 
            feature_ranks['Lasso_Coef_Norm'] + 
            feature_ranks['Correlation_Norm']
        )
        
        # Sort by combined score
        feature_ranks = feature_ranks.sort_values('Combined_Score', ascending=False)
        
        # Select top features
        top_features = feature_ranks.head(20)['Feature'].tolist()
        
        # Make sure key features are included
        for feature in key_features_present:
            if feature not in top_features:
                top_features.append(feature)
                
        print(f"Selected {len(top_features)} features using combined importance methods")
        
        # Display top 10 features with scores
        print("\nTop 10 features by combined importance:")
        top_10_display = feature_ranks.head(10)[['Feature', 'Combined_Score', 'RF_Importance', 'Lasso_Coef', 'Correlation']]
        print(top_10_display)
    
    else:
        # If no method specified, use all features
        top_features = X_train.columns.tolist()
        print(f"Using all {len(top_features)} features")
    
    # Filter datasets to selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]
    
    return X_train_selected, X_test_selected, top_features

In [10]:
# Function for model evaluation
def evaluate_models(X_train, y_train):
    """
    Evaluate multiple regression models
    """
    print("\n==== Model Evaluation ====")
    
    # Define models to evaluate
    models = {
        'SVR': SVR(),
        'RandomForest': RandomForestRegressor(random_state=42),
        'GradientBoosting': GradientBoostingRegressor(random_state=42),
        'XGBoost': XGBRegressor(random_state=42),
        'Lasso': Lasso(random_state=42),
        'Ridge': Ridge(random_state=42),
        'ElasticNet': ElasticNet(random_state=42)
    }
    
    # Cross-validation settings
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    results = {}
    
    # Evaluate each model
    for name, model in models.items():
        # Calculate cross-validation scores
        rmse_scores = np.sqrt(-cross_val_score(model, X_train, y_train, 
                                             scoring='neg_mean_squared_error', 
                                             cv=cv))
        r2_scores = cross_val_score(model, X_train, y_train, 
                                    scoring='r2', 
                                    cv=cv)
        
        # Store results
        results[name] = {
            'RMSE': rmse_scores.mean(),
            'RMSE_std': rmse_scores.std(),
            'R2': r2_scores.mean(),
            'R2_std': r2_scores.std()
        }
        
        print(f"{name}: RMSE = {rmse_scores.mean():.4f} ± {rmse_scores.std():.4f}, R² = {r2_scores.mean():.4f} ± {r2_scores.std():.4f}")
    
    # Identify best model
    best_model = min(results.items(), key=lambda x: x[1]['RMSE'])[0]
    print(f"\nBest model based on RMSE: {best_model}")
    
    return results, best_model

In [11]:
# Function for hyperparameter tuning
def tune_model(X_train, y_train, model_name):
    """
    Perform hyperparameter tuning for the selected model
    """
    print(f"\n==== Hyperparameter Tuning for {model_name} ====")
    
    if model_name == 'SVR':
        # Define search space for SVR
        param_grid = {
            'C': [0.1, 1, 10, 50, 100],
            'epsilon': [0.001, 0.01, 0.1, 0.2],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['scale', 'auto', 0.1, 0.01]
        }
        base_model = SVR()
        
    elif model_name == 'RandomForest':
        # Define search space for Random Forest
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        }
        base_model = RandomForestRegressor(random_state=42)
        
    elif model_name == 'GradientBoosting':
        # Define search space for Gradient Boosting
        param_grid = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 5, 10],
            'subsample': [0.7, 0.8, 0.9, 1.0]
        }
        base_model = GradientBoostingRegressor(random_state=42)
        
    elif model_name == 'XGBoost':
        # Define search space for XGBoost
        param_grid = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3, 5],
            'subsample': [0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
            'gamma': [0, 0.1, 0.2]
        }
        base_model = XGBRegressor(random_state=42)
        
    elif model_name == 'Lasso':
        # Define search space for Lasso
        param_grid = {
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
            'selection': ['cyclic', 'random'],
            'tol': [1e-4, 1e-3, 1e-2]
        }
        base_model = Lasso(random_state=42)
        
    elif model_name == 'Ridge':
        # Define search space for Ridge
        param_grid = {
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
            'solver': ['auto', 'svd', 'cholesky', 'lsqr'],
        }
        base_model = Ridge(random_state=42)
        
    elif model_name == 'ElasticNet':
        # Define search space for ElasticNet
        param_grid = {
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
            'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'selection': ['cyclic', 'random']
        }
        base_model = ElasticNet(random_state=42)
        
    else:
        print(f"Model {model_name} not supported for tuning")
        return None
    
    # Use RandomizedSearchCV for faster tuning with cross-validation
    search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_grid,
        n_iter=50,  # Number of parameter settings sampled
        scoring='neg_mean_squared_error',
        cv=5,
        verbose=1,
        random_state=42,
        n_jobs=-1
    )
    
    # Fit to data
    search.fit(X_train, y_train)
    
    # Get best parameters and score
    best_params = search.best_params_
    best_score = np.sqrt(-search.best_score_)
    
    print(f"Best parameters: {best_params}")
    print(f"Best RMSE: {best_score:.4f}")
    
    # Create model with best parameters
    if model_name == 'SVR':
        best_model = SVR(**best_params)
    elif model_name == 'RandomForest':
        best_model = RandomForestRegressor(random_state=42, **best_params)
    elif model_name == 'GradientBoosting':
        best_model = GradientBoostingRegressor(random_state=42, **best_params)
    elif model_name == 'XGBoost':
        best_model = XGBRegressor(random_state=42, **best_params)
    elif model_name == 'Lasso':
        best_model = Lasso(random_state=42, **best_params)
    elif model_name == 'Ridge':
        best_model = Ridge(random_state=42, **best_params)
    elif model_name == 'ElasticNet':
        best_model = ElasticNet(random_state=42, **best_params)
    
    return best_model, best_params, best_score

In [12]:
# Function to create ensemble model
def create_ensemble(X_train, y_train, models_dict):
    """
    Create an ensemble of the best models
    """
    print("\n==== Creating Ensemble Model ====")
    
    # Select models for ensemble
    ensemble_models = {}
    for model_name, model_obj in models_dict.items():
        ensemble_models[model_name] = model_obj
        
    # Train all models
    predictions = {}
    for name, model in ensemble_models.items():
        model.fit(X_train, y_train)
        predictions[name] = model.predict(X_train)
    
    # Calculate optimal weights using a simple meta-model
    # Convert predictions to DataFrame
    pred_df = pd.DataFrame(predictions)
    
    # Meta-model: Ridge regression to find optimal weights
    meta_model = Ridge(alpha=1.0)
    meta_model.fit(pred_df, y_train)
    
    # Get weights (coefficients)
    weights = meta_model.coef_
    weights = np.maximum(weights, 0)  # Ensure non-negative weights
    weights = weights / weights.sum()  # Normalize weights to sum to 1
    
    # Create weights dictionary
    weights_dict = {model_name: weight for model_name, weight in zip(ensemble_models.keys(), weights)}
    
    print("Ensemble model weights:")
    for model_name, weight in weights_dict.items():
        print(f"  {model_name}: {weight:.4f}")
    
    # Create ensemble prediction function
    def ensemble_predict(X):
        """Generate predictions using the weighted ensemble"""
        pred = np.zeros(X.shape[0])
        for name, model in ensemble_models.items():
            pred += weights_dict[name] * model.predict(X)
        return pred
    
    # Test ensemble on training data
    ensemble_train_pred = ensemble_predict(X_train)
    ensemble_rmse = np.sqrt(mean_squared_error(y_train, ensemble_train_pred))
    ensemble_r2 = r2_score(y_train, ensemble_train_pred)
    
    print(f"Ensemble Training RMSE: {ensemble_rmse:.4f}")
    print(f"Ensemble Training R²: {ensemble_r2:.4f}")
    
    # Return ensemble components
    ensemble = {
        'models': ensemble_models,
        'weights': weights_dict,
        'predict': ensemble_predict,
        'meta_model': meta_model,
        'performance': {
            'rmse': ensemble_rmse,
            'r2': ensemble_r2
        }
    }
    
    return ensemble

In [13]:
# Function to make predictions on test data
def make_predictions(models, X_test, ensemble=None):
    """
    Generate predictions on test data using individual models and ensemble
    """
    print("\n==== Making Predictions ====")
    
    # Dictionary to store predictions
    predictions = {}
    
    # Get predictions from individual models
    for name, model in models.items():
        predictions[name] = model.predict(X_test)
    
    # Add ensemble predictions if available
    if ensemble:
        predictions['Ensemble'] = ensemble['predict'](X_test)
    
    # Convert to DataFrame
    predictions_df = pd.DataFrame(predictions)
    
    return predictions_df

In [14]:
# Function to visualize model results
def visualize_model_results(results, y_train, train_predictions, top_features):
    """
    Create visualizations of model performance and feature importance
    """
    print("\n==== Visualizing Model Results ====")
    
    # 1. Model Performance Comparison
    plt.figure(figsize=(12, 6))
    
    # Extract RMSE values and create error bars
    models = list(results.keys())
    rmse_values = [results[model]['RMSE'] for model in models]
    rmse_errors = [results[model]['RMSE_std'] for model in models]
    
    # Sort by performance
    sorted_indices = np.argsort(rmse_values)
    sorted_models = [models[i] for i in sorted_indices]
    sorted_rmse = [rmse_values[i] for i in sorted_indices]
    sorted_errors = [rmse_errors[i] for i in sorted_indices]
    
    # Create bar plot
    bars = plt.bar(sorted_models, sorted_rmse, yerr=sorted_errors, 
                   color=colors['primary'], alpha=0.7, 
                   capsize=5, error_kw={'ecolor': colors['quaternary']})
    
    # Highlight best model
    best_model_idx = sorted_models.index(min(results.items(), key=lambda x: x[1]['RMSE'])[0])
    bars[best_model_idx].set_color(colors['secondary'])
    
    plt.title('Model Performance Comparison (RMSE)', fontsize=14)
    plt.xlabel('Models', fontsize=12)
    plt.ylabel('RMSE (Root Mean Squared Error)', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig('model_performance.png', dpi=300, bbox_inches='tight')
    
    # 2. Actual vs Predicted for best model
    plt.figure(figsize=(10, 6))
    
    # Get predictions from best model
    best_model_name = min(results.items(), key=lambda x: x[1]['RMSE'])[0]
    best_model_preds = train_predictions[best_model_name]
    
    # Scatterplot of actual vs predicted
    plt.scatter(y_train, best_model_preds, 
               color=colors['primary'], alpha=0.7, edgecolor='k', linewidth=0.5)
    
    # Add diagonal line (perfect predictions)
    min_val = min(min(y_train), min(best_model_preds))
    max_val = max(max(y_train), max(best_model_preds))
    plt.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5)
    
    # Add labels and title
    plt.title(f'Actual vs Predicted T80 Values ({best_model_name})', fontsize=14)
    plt.xlabel('Actual T80', fontsize=12)
    plt.ylabel('Predicted T80', fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Add R² value
    r2 = r2_score(y_train, best_model_preds)
    plt.annotate(f'R² = {r2:.4f}', 
                xy=(0.05, 0.95), xycoords='axes fraction',
                fontsize=12, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('actual_vs_predicted.png', dpi=300, bbox_inches='tight')
    
    # 3. Feature Importance (for Random Forest or XGBoost model)
    if 'RandomForest' in results or 'XGBoost' in results:
        plt.figure(figsize=(10, 8))
        
        # Get feature importance from tree-based model
        if 'RandomForest' in results:
            model_name = 'RandomForest'
        else:
            model_name = 'XGBoost'
            
        # Get model
        for name, model in train_predictions.items():
            if name == model_name:
                tree_model = model
                break
        
        # Get feature importances
        importances = tree_model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        # Plot top 15 features
        plt.barh(range(min(15, len(top_features))), 
                importances[indices[:15]], 
                color=colors['tertiary'], alpha=0.7)
        plt.yticks(range(min(15, len(top_features))), [top_features[i] for i in indices[:15]])
        plt.title(f'Feature Importance ({model_name})', fontsize=14)
        plt.xlabel('Importance', fontsize=12)
        plt.tight_layout()
        plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
    
    print("Model result visualizations saved as PNG files")
    return {
        'model_performance.png': 'Comparison of model performance (RMSE)',
        'actual_vs_predicted.png': 'Actual vs Predicted values for best model',
        'feature_importance.png': 'Feature importance from tree-based model'
    }

In [15]:
# Function to train and save final model
def train_final_model(X_train, y_train, best_model_type, best_params):
    """
    Train the final model with the best parameters and save it
    """
    print("\n==== Training Final Model ====")
    
    # Create model with best parameters
    if best_model_type == 'SVR':
        final_model = SVR(**best_params)
    elif best_model_type == 'RandomForest':
        final_model = RandomForestRegressor(random_state=42, **best_params)
    elif best_model_type == 'GradientBoosting':
        final_model = GradientBoostingRegressor(random_state=42, **best_params)
    elif best_model_type == 'XGBoost':
        final_model = XGBRegressor(random_state=42, **best_params)
    elif best_model_type == 'Lasso':
        final_model = Lasso(random_state=42, **best_params)
    elif best_model_type == 'Ridge':
        final_model = Ridge(random_state=42, **best_params)
    elif best_model_type == 'ElasticNet':
        final_model = ElasticNet(random_state=42, **best_params)
    else:
        print(f"Model type {best_model_type} not supported")
        return None
    
    # Train model on full data
    final_model.fit(X_train, y_train)
    
    # Save model
    with open('final_model.pkl', 'wb') as f:
        pickle.dump(final_model, f)
    
    print(f"Final {best_model_type} model trained and saved as 'final_model.pkl'")
    
    return final_model

In [16]:
# Function to save project components
def save_project_components(scaler, selected_features, feature_engineering_info, best_model_type, best_params):
    """
    Save all necessary components for model deployment
    """
    print("\n==== Saving Project Components ====")
    
    components = {
        'scaler': scaler,
        'selected_features': selected_features,
        'feature_engineering_info': feature_engineering_info,
        'best_model_type': best_model_type,
        'best_params': best_params,
        'model_date': pd.Timestamp.now().strftime('%Y-%m-%d')
    }
    
    with open('model_components.pkl', 'wb') as f:
        pickle.dump(components, f)
    
    print("Project components saved as 'model_components.pkl'")
    
    return components

In [17]:
# Main function to run the whole pipeline
def main_pipeline(train_path="../molecular-machine-learning/data/train.csv", 
                 test_path="../molecular-machine-learning/data/test.csv",
                 submission_path="../molecular-machine-learning/data/sample_submission.csv"):
    """
    End-to-end pipeline for molecular photostability prediction
    """
    print("\n==== MOLECULAR PHOTOSTABILITY PREDICTION PIPELINE ====")
    
    # 1. Load data
    df_train, df_test, submission = load_data(train_path, test_path)
    
    # 2. Explore data
    data_info = explore_data(df_train)
    
    # 3. Clean data
    df_train_clean, df_test_clean = clean_data(df_train, df_test)
    
    # 4. Engineer features
    df_train_enhanced, df_test_enhanced = engineer_rdkit_features(df_train_clean, df_test_clean)
    
    # 5. Create visualizations
    visualizations = visualize_data(df_train_enhanced)
    
    # 6. Prepare data for modeling
    # Split features and target
    X_train = df_train_enhanced.drop(['T80', 'Batch_ID', 'Smiles'], axis=1, errors='ignore')
    y_train = df_train_enhanced['T80']
    X_test = df_test_enhanced.drop(['Batch_ID', 'Smiles'], axis=1, errors='ignore')
    
    # 7. Select features
    X_train_selected, X_test_selected, selected_features = select_features(X_train, y_train, X_test, method='combined')
    
    # 8. Scale features
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train_selected)
    X_test_scaled = scaler.transform(X_test_selected)
    
    # Convert back to DataFrame to keep feature names
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_selected.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_selected.columns)
    
    # 9. Evaluate models
    model_results, best_model_type = evaluate_models(X_train_scaled_df, y_train)
    
    # 10. Tune best model
    best_model, best_params, best_score = tune_model(X_train_scaled_df, y_train, best_model_type)
    
    # 11. Train individual models with best parameters
    models = {}
    
    # Best SVR model
    svr_params = {'C': 10, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'rbf'} if best_model_type != 'SVR' else best_params
    models['SVR'] = SVR(**svr_params).fit(X_train_scaled_df, y_train)
    
    # Best Random Forest model
    rf_params = {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200} if best_model_type != 'RandomForest' else best_params
    models['RandomForest'] = RandomForestRegressor(random_state=42, **rf_params).fit(X_train_scaled_df, y_train)
    
    # Best XGBoost model
    xgb_params = {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 5, 'subsample': 0.9} if best_model_type != 'XGBoost' else best_params
    models['XGBoost'] = XGBRegressor(random_state=42, **xgb_params).fit(X_train_scaled_df, y_train)
    
    # Best Lasso model
    lasso_params = {'alpha': 0.01} if best_model_type != 'Lasso' else best_params
    models['Lasso'] = Lasso(random_state=42, **lasso_params).fit(X_train_scaled_df, y_train)
    
    # 12. Create ensemble
    ensemble = create_ensemble(X_train_scaled_df, y_train, models)
    
    # 13. Generate training predictions for visualization
    train_predictions = {name: model.predict(X_train_scaled_df) for name, model in models.items()}
    train_predictions['Ensemble'] = ensemble['predict'](X_train_scaled_df)
    
    # 14. Visualize model results
    model_visualizations = visualize_model_results(model_results, y_train, models, selected_features)
    
    # 15. Make predictions on test data
    test_predictions_df = make_predictions(models, X_test_scaled_df, ensemble)
    
    # 16. Create submission file with ensemble predictions
    submission['T80'] = ensemble['predict'](X_test_scaled_df)
    submission.to_csv('ensemble_submission.csv', index=False)
    
    # 17. Train and save final model
    final_model = train_final_model(X_train_scaled_df, y_train, best_model_type, best_params)
    
    # 18. Save project components
    feature_engineering_info = {
        'key_features': ['TDOS4.0', 'NumHeteroatoms', 'Mass'],
        'rdkit_features': True,
        'cleaning_applied': True
    }
    project_components = save_project_components(scaler, selected_features, feature_engineering_info, best_model_type, best_params)
    
    print("\n==== Pipeline Complete ====")
    print(f"Best model: {best_model_type}")
    print(f"Best RMSE: {best_score:.4f}")
    print(f"Selected features: {len(selected_features)}")
    print("Submission file saved as 'ensemble_submission.csv'")
    
    return {
        'train_data': df_train_enhanced,
        'test_data': df_test_enhanced,
        'selected_features': selected_features,
        'scaler': scaler,
        'models': models,
        'ensemble': ensemble,
        'best_model': final_model,
        'best_model_type': best_model_type,
        'best_params': best_params,
        'submission': submission,
        'visualizations': {**visualizations, **model_visualizations}
    }